In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.stattools as stattools
from scipy.stats import mstats
from scipy.stats import zscore
from scipy.stats import shapiro


# Lendo a planilha com os dados 

In [2]:
def ler_planilha (path):
    df = pd.read_excel(path, sheet_name='Dados')
    df = df.set_index("Replicatas") #df.set_index("Replicatas", inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')# Converter as colunas para formato numérico, se necessário
    df = df.round(2)
    return df

In [3]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

In [4]:
df

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
Replicatas,,,,,,
1,73439.33,114422.95,137847.37,168777.89,219072.22,281650.94
2,74709.57,114855.20,136493.41,172028.96,222644.05,276065.01
3,75383.99,116134.25,135820.57,172089.50,222438.49,277585.04
4,76085.28,117401.73,136135.26,173948.81,223538.30,278630.83
5,76761.00,117268.61,137848.39,175236.13,220192.94,281322.57
6,76516.32,117733.62,138811.59,175135.40,220294.16,280352.98
7,76067.53,115657.58,138185.08,175373.97,220294.16,281291.21


In [5]:
#df.describe()

In [6]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# Calcular a média e desvio padrão de cada coluna
medias = df.mean().round(2)
desvios_padrao = df.std().round(2)
variancias = df.var().round(2)

print(medias)
print(desvios_padrao)
print(variancias)


Padrão 1     75566.15
Padrão 2    116210.56
Padrão 3    137305.95
Padrão 4    173227.24
Padrão 5    221210.62
Padrão 6    279556.94
dtype: float64
Padrão 1    1164.88
Padrão 2    1303.98
Padrão 3    1144.86
Padrão 4    2426.69
Padrão 5    1646.65
Padrão 6    2163.46
dtype: float64
Padrão 1    1356940.38
Padrão 2    1700371.31
Padrão 3    1310714.70
Padrão 4    5888811.54
Padrão 5    2711450.91
Padrão 6    4680550.31
dtype: float64


In [7]:
dfmin = df.min()

In [8]:
dfmin

Padrão 1     73439.33
Padrão 2    114422.95
Padrão 3    135820.57
Padrão 4    168777.89
Padrão 5    219072.22
Padrão 6    276065.01
dtype: float64

In [9]:
# Calcular os valores normalizados Gmin e Gmax
Gmin = ((medias - df.min()) / desvios_padrao).round(3)
Gmax = ((df.max() - medias) / desvios_padrao).round(3)

In [10]:
Gmin

Padrão 1    1.826
Padrão 2    1.371
Padrão 3    1.297
Padrão 4    1.834
Padrão 5    1.299
Padrão 6    1.614
dtype: float64

In [11]:
Gmax

Padrão 1    1.026
Padrão 2    1.168
Padrão 3    1.315
Padrão 4    0.885
Padrão 5    1.414
Padrão 6    0.968
dtype: float64

In [12]:
# Criar um DataFrame com as estatísticas calculadas
estatisticas_df = pd.DataFrame([medias, desvios_padrao, Gmin, Gmax, variancias], 
                               index=['Média', 'Stand. Dev.', 'Gmin', 'Gmax', 'Variância'])

# Adicionar o DataFrame de estatísticas ao final do DataFrame original
df = pd.concat([df, estatisticas_df])


df

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
1,73439.330,114422.950,137847.370,168777.890,219072.220,281650.940
2,74709.570,114855.200,136493.410,172028.960,222644.050,276065.010
3,75383.990,116134.250,135820.570,172089.500,222438.490,277585.040
4,76085.280,117401.730,136135.260,173948.810,223538.300,278630.830
5,76761.000,117268.610,137848.390,175236.130,220192.940,281322.570
6,76516.320,117733.620,138811.590,175135.400,220294.160,280352.980
7,76067.530,115657.580,138185.080,175373.970,220294.160,281291.210
Média,75566.150,116210.560,137305.950,173227.240,221210.620,279556.940
Stand. Dev.,1164.880,1303.980,1144.860,2426.690,1646.650,2163.460
Gmin,1.826,1.371,1.297,1.834,1.299,1.614


# Teste Shapiro Wilk

In [13]:
def shapiro_teste(col):
    stat, p = shapiro(col)
    if p > 0.05:
        return True
    else:
        return False

def shapiro_estatistica(col):
    stat, p = shapiro(col)
    return stat

def shapiro_p(col):
    stat, p = shapiro(col)
    return p

def shapiro_df(df):
    estatisticas_shapiro = df.apply(shapiro_estatistica)
    estatisticas_df = pd.DataFrame([estatisticas_shapiro], index=['Estatística W'])
    df = pd.concat([df, estatisticas_df])

    p_shapiro = df.apply(shapiro_p)
    p_df = pd.DataFrame([p_shapiro], index=['Valor p'])
    df = pd.concat([df, p_df])

    normal_shapiro = df.apply(shapiro_teste).apply(lambda x: "Sim" if x else "Não")
    normal_df = pd.DataFrame([normal_shapiro], index=['Distribuição Normal'])
    df = pd.concat([df, normal_df])
    return df

In [17]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')
resultados = shapiro_df(df)
resultados

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
1,73439.33,114422.95,137847.37,168777.89,219072.22,281650.94
2,74709.57,114855.2,136493.41,172028.96,222644.05,276065.01
3,75383.99,116134.25,135820.57,172089.5,222438.49,277585.04
4,76085.28,117401.73,136135.26,173948.81,223538.3,278630.83
5,76761.0,117268.61,137848.39,175236.13,220192.94,281322.57
6,76516.32,117733.62,138811.59,175135.4,220294.16,280352.98
7,76067.53,115657.58,138185.08,175373.97,220294.16,281291.21
Estatística W,0.906179,0.918947,0.910628,0.859808,0.900818,0.886165
Valor p,0.000003,0.000002,0.000002,0.000003,0.000002,0.000002
Distribuição Normal,Não,Não,Não,Não,Não,Não


In [18]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

from scipy.stats import shapiro

for col in df.columns:
    stat, p = shapiro(df[col])
    print(f'Coluna {col}:')
    print('Estatística de teste:', stat)
    print('Valor p:', p)
    if p > 0.05:
        print('Os dados parecem seguir uma distribuição normal.\n')
    else:
        print('Os dados não seguem uma distribuição normal.\n')

Coluna Padrão 1:
Estatística de teste: 0.9061791896820068
Valor p: 0.3700520992279053
Os dados parecem seguir uma distribuição normal.

Coluna Padrão 2:
Estatística de teste: 0.9189468622207642
Valor p: 0.4612550437450409
Os dados parecem seguir uma distribuição normal.

Coluna Padrão 3:
Estatística de teste: 0.9106277823448181
Valor p: 0.40023767948150635
Os dados parecem seguir uma distribuição normal.

Coluna Padrão 4:
Estatística de teste: 0.8598077893257141
Valor p: 0.15079085528850555
Os dados parecem seguir uma distribuição normal.

Coluna Padrão 5:
Estatística de teste: 0.9008180499076843
Valor p: 0.3359634280204773
Os dados parecem seguir uma distribuição normal.

Coluna Padrão 6:
Estatística de teste: 0.8861648440361023
Valor p: 0.2552492916584015
Os dados parecem seguir uma distribuição normal.



# Teste Grubbs

In [19]:
# Teste de outliers (Grubbs)
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

def grubbs_test(data):
    mean = data.mean()
    std_dev = data.std()
    n = len(data)

    max_outlier = max(data)
    min_outlier = min(data)

    z = (max_outlier - mean) / std_dev if max_outlier > mean else (min_outlier - mean) / std_dev

    critical_value = stats.t.ppf(1 - 0.05 / (2 * n), n - 2)

    return z > critical_value

for col in df.columns:
    is_outlier = grubbs_test(df[col])
    print(f'Coluna {col}:')
    if is_outlier:
        print('Outlier encontrado.\n')
    else:
        print('Nenhum outlier encontrado.\n')

Coluna Padrão 1:
Nenhum outlier encontrado.

Coluna Padrão 2:
Nenhum outlier encontrado.

Coluna Padrão 3:
Nenhum outlier encontrado.

Coluna Padrão 4:
Nenhum outlier encontrado.

Coluna Padrão 5:
Nenhum outlier encontrado.

Coluna Padrão 6:
Nenhum outlier encontrado.



# Teste Cochran

In [20]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# Teste de homocedasticidade (Cochran)
stat, pvalue = stats.levene(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Os dados não são homocedásticos.')
else:
    print('Os dados são homocedásticos.')



Os dados são homocedásticos.


# Teste Anova

In [21]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# ANOVA (falta de ajuste e erro puro)
F, pvalue = stats.f_oneway(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Existe uma diferença significativa entre os grupos.')
else:
    print('Não existe uma diferença significativa entre os grupos.')

Não existe uma diferença significativa entre os grupos.


# Durbin Watson

In [22]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# Teste de independência dos resíduos (Durbin Watson)
# Obter o teste de Durbin-Watson
dw = stattools.durbin_watson(df['Padrão 1'])

# Imprimir o resultado do teste
print(dw)

8.199227618926422e-05
